In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier
import math

In [2]:
iris = load_iris()

In [3]:
#iris.data -> features, iris.target -> labels
X = iris.data
y = iris.target

- About data : https://www.kaggle.com/uciml/iris

In [4]:
# split into test and train dataset, and use random_state=48
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=48)

In [5]:
from sklearn.preprocessing import StandardScaler

- Documentation for "StandardScaler" : https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [6]:
ss = StandardScaler()

X_train=ss.fit_transform(X_train)
X_test=ss.transform(X_test)

In [8]:
X_train

array([[-0.26019744, -0.64203312,  0.6756601 ,  1.05971782],
       [ 2.09117695,  1.61259823,  1.65645703,  1.31289273],
       [-0.48413786,  1.83806137, -1.06797887, -0.96568143],
       [-0.93201869,  0.93620883, -1.28593375, -1.09226888],
       [ 1.53132591,  0.25981942,  1.275036  ,  0.80654291],
       [-0.82004848,  1.61259823, -0.95900144, -0.96568143],
       [ 0.85950465, -0.19110685,  0.40321651,  0.3001931 ],
       [-1.15595911,  0.71074569, -0.95900144, -1.21885633],
       [-0.59610806,  1.3871351 , -1.17695631, -1.21885633],
       [-0.48413786,  1.3871351 , -1.17695631, -1.21885633],
       [-0.03625702, -0.86749625,  0.78463754,  0.93313037],
       [-1.26792932,  0.25981942, -1.12246759, -1.21885633],
       [ 1.19541528,  0.03435629,  0.94810369,  1.18630527],
       [-1.0439889 ,  0.03435629, -1.17695631, -1.34544378],
       [-1.0439889 , -1.31842252,  0.45770523,  0.67995546],
       [ 0.41162382, -0.41656998,  0.3487278 ,  0.17360565],
       [ 1.53132591,  1.

In [16]:
trainSet=np.column_stack((X_train, y_train))
testSet=np.column_stack((X_test, y_test))

### KNN Classifier Implementation

In [242]:
import math
def getDistance(p, instance1, instance2):
    # p=1 : return Manhattan Distance
    # p=2 : return Eucludean Distance
    # 데이터에 결과값은 거리 계산에 포함하면 안됨
    instance1 = instance1[:-1]
    instance2= instance2[:-1]
    
    # 그냥 절대값 거리를 구하는 맨해턴 distance
    if p==1:
        distance = sum(abs(a-b) for a, b in zip(instance1[:-1], instance2[:-1]))
    # 유클라디안 distance
    elif p==2:
        distance = math.sqrt(sum(pow(a-b,2) for a, b in zip(instance1[:-1], instance2[:-1])))
    
    return distance

In [243]:
def getNeighbors(p, trainSet, testInstance, k):
    # p=1 : return Manhattan Distance
    # p=2 : return Eucludean Distance
    # Test 한 점에서 모든 trainSet 거리 구하기
    min_neighbors_dist = [float("inf")]*k # 일단 모두 최대값으로 두기
    min_neighbors_idx = [float("inf")]*k
    for i, one_train in enumerate(trainSet):
        distance = getDistance(p, one_train, testInstance)
        # max보다 작은 애면 일단 max 바꿔끼우기
        if distance < max(min_neighbors_dist):
            # max 값이 있는 자리에 해당 값 넣기
            min_neighbors_dist[min_neighbors_dist.index(max(min_neighbors_dist))] = distance
            min_neighbors_idx[min_neighbors_dist.index(max(min_neighbors_dist))] = i

    return min_neighbors_idx

In [244]:
import operator
from collections import Counter

def getResponse(trainSet, neighbors):
    '''
    어떤거 고를지 뽑기
    '''
    neighbor_data = []
    # 실제 trainSet 데이터 얻기
    for neighbor in neighbors:
        neighbor_data.append(trainSet[neighbor])

    neighbor_label = list(map(lambda x: x[-1],neighbor_data))
    x = Counter(neighbor_label)
    max_res = x.most_common(1)[0][0] # 가장 자주 나온 애 class 얻기
    
    # wieght vote
    
    return max_res


In [281]:
import operator
from collections import Counter

def getWeightResponse(trainSet, neighbors, test):
    '''
    어떤거 고를지 뽑기
    '''
    neighbor_data = []
    similarity_data = []
    # similarity 구하기
    for neighbor in neighbors:
        neighbor_data.append(trainSet[neighbor])
        print(trainSet[neighbor][-1],1/getDistance(2,trainSet[neighbor],test))
        similarity_data.append(1/getDistance(2,trainSet[neighbor],test))
        
#     print(similarity_data)
    # class별로 가능성 찾기
    class_dict = {}
    for i,one_neighbor_data in enumerate(neighbor_data):
        if one_neighbor_data[-1] in class_dict:
            class_dict[one_neighbor_data[-1]] += similarity_data[i]/sum(similarity_data)
        else:
            class_dict[one_neighbor_data[-1]] = similarity_data[i]/sum(similarity_data)
    print(class_dict)
    max_p = 0
    max_class = ''
    for cl in class_dict:
        print(cl)
        if max_p < class_dict[cl]:
            max_p = class_dict[cl]
            max_class = cl
    
    return max_class


In [265]:
def getAccuracy(testSet, predictions):
    '''
    result 
    '''
    correct_cnt = 0
    for i in range(len(testSet)):
#         print(testSet[i][-1],predictions[i])
        if testSet[i][-1] == predictions[i]:
            correct_cnt +=1
    accuracy_score = correct_cnt/len(predictions)
    
    return accuracy_score

In [266]:
k = 3
p = 2 # Euclidean distance

# prediction

## weight 없는 prediction

#### 맨해턴 거리

In [289]:
predictions=[]
p=1
for i in range(len(testSet)):
    neighbors = getNeighbors(p, trainSet, testSet[i], k)
#     print(neighbors)
    result = getResponse(trainSet, neighbors)
#     result = getWeightResponse(trainSet, neighbors,testSet)
    predictions.append(result)
    print(str(i) + ' > predicted : ' + str(result) + ', actual : ' + str(testSet[i][-1]))

0 > predicted : 1.0, actual : 1.0
1 > predicted : 2.0, actual : 1.0
2 > predicted : 2.0, actual : 2.0
3 > predicted : 0.0, actual : 0.0
4 > predicted : 1.0, actual : 1.0
5 > predicted : 2.0, actual : 2.0
6 > predicted : 0.0, actual : 0.0
7 > predicted : 1.0, actual : 2.0
8 > predicted : 0.0, actual : 0.0
9 > predicted : 1.0, actual : 1.0
10 > predicted : 2.0, actual : 2.0
11 > predicted : 0.0, actual : 0.0
12 > predicted : 0.0, actual : 0.0
13 > predicted : 1.0, actual : 2.0
14 > predicted : 1.0, actual : 1.0
15 > predicted : 1.0, actual : 1.0
16 > predicted : 0.0, actual : 0.0
17 > predicted : 1.0, actual : 1.0
18 > predicted : 2.0, actual : 1.0
19 > predicted : 2.0, actual : 2.0
20 > predicted : 0.0, actual : 0.0
21 > predicted : 2.0, actual : 2.0
22 > predicted : 2.0, actual : 1.0
23 > predicted : 1.0, actual : 1.0
24 > predicted : 2.0, actual : 2.0
25 > predicted : 0.0, actual : 0.0
26 > predicted : 0.0, actual : 0.0
27 > predicted : 2.0, actual : 2.0
28 > predicted : 2.0, actual :

In [290]:
accuracy = getAccuracy(testSet, predictions)
print('Accuracy: ' + str(accuracy) + '%')

Accuracy: 0.8444444444444444%


#### 유클리디안 거리

In [293]:
predictions=[]
p=2
for i in range(len(testSet)):
    neighbors = getNeighbors(p, trainSet, testSet[i], k)
#     print(neighbors)
    result = getResponse(trainSet, neighbors)
#     result = getWeightResponse(trainSet, neighbors,testSet)
    predictions.append(result)
    print(str(i) + ' > predicted : ' + str(result) + ', actual : ' + str(testSet[i][-1]))

0 > predicted : 1.0, actual : 1.0
1 > predicted : 1.0, actual : 1.0
2 > predicted : 1.0, actual : 2.0
3 > predicted : 1.0, actual : 0.0
4 > predicted : 1.0, actual : 1.0
5 > predicted : 2.0, actual : 2.0
6 > predicted : 0.0, actual : 0.0
7 > predicted : 1.0, actual : 2.0
8 > predicted : 0.0, actual : 0.0
9 > predicted : 1.0, actual : 1.0
10 > predicted : 2.0, actual : 2.0
11 > predicted : 0.0, actual : 0.0
12 > predicted : 0.0, actual : 0.0
13 > predicted : 1.0, actual : 2.0
14 > predicted : 1.0, actual : 1.0
15 > predicted : 1.0, actual : 1.0
16 > predicted : 0.0, actual : 0.0
17 > predicted : 1.0, actual : 1.0
18 > predicted : 2.0, actual : 1.0
19 > predicted : 2.0, actual : 2.0
20 > predicted : 0.0, actual : 0.0
21 > predicted : 2.0, actual : 2.0
22 > predicted : 2.0, actual : 1.0
23 > predicted : 1.0, actual : 1.0
24 > predicted : 2.0, actual : 2.0
25 > predicted : 0.0, actual : 0.0
26 > predicted : 0.0, actual : 0.0
27 > predicted : 2.0, actual : 2.0
28 > predicted : 2.0, actual :

In [294]:
accuracy = getAccuracy(testSet, predictions)
print('Accuracy: ' + str(accuracy) + '%')

Accuracy: 0.7555555555555555%


-> 유클리디안 거리. 맨해튼 거리 비교

https://www.quora.com/What-is-the-difference-between-Manhattan-and-Euclidean-distance-measures

유클리디안 거리는 차원이 커지고 거리가 멀어질수록 더 값이 커진다. 그런데 이 거리가 차원이 커질수록 값이 더 커지기 때문에 high dimension에서는 맨해튼 거리가 더 좋을수도 있다.
?? 그렇다면 왜 맨해튼 거리를 안쓰고 유클리디안 거리를 쓰는가? -> 좀더 직관적이라서 그렇다고 한다

그래서 맨해튼 거리가 더 accuracy가 높다고 생각해볼 수 있다.

## Wieght 활용한 선택

### 맨해턴 거리

In [295]:
p = 1
predictions=[]

for i in range(len(testSet)):
    neighbors = getNeighbors(p, trainSet, testSet[i], k)
#     print(neighbors)
    result = getWeightResponse(trainSet, neighbors,testSet[i])
    predictions.append(result)
    print(str(i) + ' > predicted : ' + str(result) + ', actual : ' + str(testSet[i][-1]))
    


1.0 4.435314886294944
1.0 2.893728072543042
1.0 1.9075400157752722
{1.0: 1.0}
1.0
0 > predicted : 1.0, actual : 1.0
2.0 2.4973458304468727
2.0 3.2000428114883186
1.0 3.003188806160204
{2.0: 0.6548287945166653, 1.0: 0.3451712054833348}
2.0
1.0
1 > predicted : 2.0, actual : 1.0
2.0 1.0354163712097497
1.0 2.6958539800527106
2.0 1.7784405860810966
{2.0: 0.51070863595024, 1.0: 0.48929136404976}
2.0
1.0
2 > predicted : 2.0, actual : 2.0
0.0 0.6258718116659794
0.0 0.6092111515033277
2.0 0.5388343515553198
{0.0: 0.6962460724168728, 2.0: 0.3037539275831272}
0.0
2.0
3 > predicted : 0.0, actual : 0.0
1.0 1.9412539736261982
1.0 1.7145966423619812
2.0 1.6061097973419176
{1.0: 0.6947696920575137, 2.0: 0.3052303079424862}
1.0
2.0
4 > predicted : 1.0, actual : 1.0
2.0 6.117474320623839
2.0 2.137646086463475
2.0 3.6067842384426902
{2.0: 0.9999999999999999}
2.0
5 > predicted : 2.0, actual : 2.0
0.0 1.4405034754245283
0.0 4.435314886294965
0.0 18.35242296187144
{0.0: 1.0}
0.0
6 > predicted : 0.0, actual 

In [296]:
accuracy = getAccuracy(testSet, predictions)
print('Accuracy: ' + str(accuracy) + '%')

Accuracy: 0.8444444444444444%


### 유클리디안 거리

In [297]:
p =2
predictions=[]

for i in range(len(testSet)):
    neighbors = getNeighbors(p, trainSet, testSet[i], k)
#     print(neighbors)
    result = getWeightResponse(trainSet, neighbors,testSet[i])
    predictions.append(result)
    print(str(i) + ' > predicted : ' + str(result) + ', actual : ' + str(testSet[i][-1]))
    


1.0 3.331628409411292
1.0 4.435314886294944
1.0 2.0303029989391503
{1.0: 1.0}
1.0
0 > predicted : 1.0, actual : 1.0
1.0 3.003188806160204
1.0 3.0031888061602157
2.0 3.2000428114883186
{1.0: 0.6524118317242298, 2.0: 0.3475881682757701}
1.0
2.0
1 > predicted : 1.0, actual : 1.0
2.0 1.7784405860810966
1.0 1.196945508221498
1.0 2.6958539800527106
{2.0: 0.31358936718672875, 1.0: 0.6864106328132713}
2.0
1.0
2 > predicted : 1.0, actual : 2.0
1.0 0.5625257808247344
0.0 0.6092111515033277
1.0 0.8336115323160315
{1.0: 0.6962068377420632, 0.0: 0.3037931622579369}
1.0
0.0
3 > predicted : 1.0, actual : 0.0
2.0 1.6061097973419176
1.0 1.7077392309044053
1.0 1.9412539736261982
{2.0: 0.3056286045715225, 1.0: 0.6943713954284776}
2.0
1.0
4 > predicted : 1.0, actual : 1.0
2.0 3.6067842384426902
2.0 2.9769823285213772
2.0 6.117474320623839
{2.0: 1.0}
2.0
5 > predicted : 2.0, actual : 2.0
0.0 1.4405034754245283
0.0 4.435314886294965
0.0 18.35242296187144
{0.0: 1.0}
0.0
6 > predicted : 0.0, actual : 0.0
1.0 

In [298]:
accuracy = getAccuracy(testSet, predictions)
print('Accuracy: ' + str(accuracy) + '%')

Accuracy: 0.7555555555555555%


-> 결과값을 살펴보면

similarity가 아래와 가팅 나왔는데

2.0 1.0354163712097497
1.0 2.6958539800527106
2.0 1.7784405860810966

1은 한번밖에 안나왔음에도 similarity가 높아서 거의 비등비등한 확률로 나타난다.

{2.0: 0.51070863595024, 1.0: 0.48929136404976}

-> 그래도 확률에 영향을 줄 정도는 아니다(accurarcy가 변하지 않음)